# **Community Detection**
----

### 1. Environment Setup
Click on the button below to open the notebook in Google Colab and run the following commands to install the required libraries.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/hardness1020/Community_Detection/blob/main/main.ipynb)

In [ ]:
!git clone https://github.com/hardness1020/Community_Detection.git

In [ ]:
# For colab
%cd Community_Detection
%pip install -r requirements.txt
!git clone https://github.com/6lyc/CDNMF.git

# For local
# %pip install -r requirements.txt
!git clone https://github.com/6lyc/CDNMF.git

### 2. Methods

#### 2-1. Spactral Clustering Method

In [ ]:
import os
import random
import numpy as np
import linecache
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
from datetime import datetime
from CDNMF.Utils.evaluate import clusterscores
from CDNMF.Dataset.dataset import Dataset
from CDNMF.Model.my_model import Model
from CDNMF.PreTrainer.pretrainer import PreTrainer
from CDNMF.Utils import gpu_info

In [ ]:
# Check GPU
if torch.cuda.is_available():
    gpu_info.get_info()
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

In [ ]:
def train(model: Model, graph, optimizer, iteration):
    optimizer.zero_grad()
    V = model()

    loss, loss1, loss2, loss3, loss4, loss5 = model.loss(graph)
    loss.backward()
    optimizer.step()

    y_pred = np.argmax(V.detach().cpu().numpy(), axis=0)
    y_true = graph.L.detach().cpu().numpy()
    # print(y_pred)
    # scores = clusterscores(y_pred, y_true)
    
    # write y_pred to file
    time = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
    cora_pred_dir = f'./prediction/cora/{iteration}/'
    if not os.path.exists(cora_pred_dir):
        os.makedirs(cora_pred_dir)
    with open(f'{cora_pred_dir}{time}.txt', 'w') as f:
        for i in y_pred:
            f.write(str(i) + '\n')

    return loss.item(), loss1.item(), loss2.item(), loss3.item(), loss4.item(), loss5.item()

In [ ]:
random.seed(42)

dataset_config = {'feature_file': './dataset/cora/features.txt',
                  'graph_file': './dataset/cora/edges.txt',
                  'walks_file': './dataset/cora/walks.txt',
                  'label_file': './dataset/cora/group.txt',
                  'device': device}
graph = Dataset(dataset_config)

pretrain_config = {
    'net_shape': [1000, 500, 7],
    'att_shape': [500, 200, 7],
    'net_input_dim': graph.num_nodes,
    'att_input_dim': graph.num_feas,
    'seed': 42,
    'pre_iterations': 100,
    'pretrain_params_path': './CDNMF/Log/cora/pretrain_params.pkl'}

model_config = {
    'device': device,
    'net_shape': [1000, 500, 7],
    'att_shape': [500, 200, 7],
    'net_input_dim': graph.num_nodes,
    'att_input_dim': graph.num_feas,
    'is_init': True,
    'pretrain_params_path': './CDNMF/Log/cora/pretrain_params.pkl',
    'tau': 1.3,
    'conc': 5,
    'negc': 400,
    'rec': 1,
    'r': 3,
    'learning_rate': 0.01,
    'weight_decay': 0.00001,
    'epoch': 550,
    'run': 20,
    'model_path': './CDNMF/Log/cora/cora_model.pkl'
}

# 'Pre-training stage'
# pretrainer = PreTrainer(pretrain_config)
# pretrainer.pre_training(graph.A.detach().cpu().numpy(), 'net')
# pretrainer.pre_training(graph.X.t().detach().cpu().numpy(), 'att')

learning_rate = model_config['learning_rate']
weight_decay = model_config['weight_decay']

M = []
N = []

# 'Fine-tuning stage'
for i in range(20):
    model = Model(model_config).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in tqdm(range(model_config['epoch'])):
        loss, loss1, loss2, loss3, loss4, loss5, scores = train(model, graph, optimizer, i)
        
#     M.append(scores['ACC'])
#     N.append(scores['NMI'])
#     # print(acc)
# print('ACC: ', np.mean(M), '; NMI: ', np.mean(N))
# print("=== Final ===")

#### 2-2. CDNMF Method